# Reaction stoichiometry (branching ratio) example

In this notebook, a branching ratio will be applied to the reaction: `A + B --> C + D` to afford `A + B --> (0.75)C + (0.25)D` with the branching ratios in brackets. 

Model construction is dealt with in the crash course notebook. Here we will focus on incorporating a branching ratio into the reaction scheme.

In [ ]:
# importing the necessaries
import numpy as np
import multilayerpy
import multilayerpy.build as build 
import multilayerpy.simulate as simulate
import multilayerpy.optimize as optimize


# useful to know what version of multilayerpy we are using
print(multilayerpy.__version__)

### Defining the reaction scheme with a branching ratio
The first thing to do is define the reaction scheme. In this case, `A + B --> (0.75)C + (0.25)D`. There are 4 components. 

It is possible to define branching ratos for both reactants and products. In this case, we are only interested in the products. In addition to the `product_tuple_list` supplied to the `ReactionScheme` object, we will define a `product_stoich` (product stoichiometry) tuple list in a similar fashion:

`product_stoich = [(0.75,0.25)]`

This states that for reaction 1, the products 3 and 4 (`C` and `D`) have stoichiometric coefficients (branching ratios) of 0.75 and 0.25, respectively.

This is then supplied to the `ReactionScheme` object as outlined in the code cell below:

In [ ]:
# import the ModelType class
from multilayerpy.build import ModelType

# import the ReactionScheme class
from multilayerpy.build import ReactionScheme

# define the model type (KM-SUB in this case) and geometry (spherical or film)
mod_type = ModelType('km-sub','spherical')

# build the reaction tuple list, in this case only 1 tuple in the list (for 1 reaction)
reaction_tuple_list = [(1,2)]


# build the product tuple list
product_tuple_list = [(3,4)]

# Now add the product stoichiometry **********
product_stoich = [(0.75,0.25)]

# now construct the reaction scheme with the product stoichiometry
# we can give it a name and define the nuber of components as below
reaction_scheme = ReactionScheme(mod_type,
                                 name='AB_CD',
                                 reactants=reaction_tuple_list,
                                 products=product_tuple_list,
                                 product_stoich=product_stoich)

# let's print out a representation of the reaction scheme
reaction_scheme.display()

In [ ]:
# Model components

# import ModelComponent class
from multilayerpy.build import ModelComponent

# making model components

# A
A = ModelComponent(1,reaction_scheme,name='A')

# B (gas)
B = ModelComponent(2,reaction_scheme,gas=True,name='B') 

# C
C = ModelComponent(3,reaction_scheme, name='C')

# D
D = ModelComponent(4,reaction_scheme, name='D')

# collect into a dictionary
model_components_dict = {'1':A,
                        '2':B,
                        '3':C,
                        '4':D}

In [ ]:
# Diffusion

# import DiffusionRegime class
from multilayerpy.build import DiffusionRegime

# making the diffusion dictionary
diff_dict = None  

# make diffusion regime
diff_regime = DiffusionRegime(mod_type,model_components_dict,diff_dict=diff_dict)

# call it to build diffusion code ready for the builder
diff_regime()


In [ ]:
# Construct the model

# import ModelBuilder class
from multilayerpy.build import ModelBuilder

# create the model object
model = ModelBuilder(reaction_scheme,model_components_dict,diff_regime)

# build the model. Will save a file, won't include the date in the model filename
model.build(date=False)

# print out the parameters required for the model to run (useful for constructing the parameter dictionary later)
print(model.req_params)

In [ ]:
# run simulation and plot

# import the Simulate class
from multilayerpy.simulate import Simulate

# import the Parameter class
from multilayerpy.build import Parameter

# make the parameter dictionary
param_dict = {'delta_3':Parameter(1e-7),
              'alpha_s_0_2':Parameter(4.2e-4),
              'delta_2':Parameter(0.4e-7),
              'Db_2':Parameter(1e-5),
              'delta_1':Parameter(0.8e-7),
              'Db_1':Parameter(1e-10),
              'Db_3':Parameter(1e-10),
              'k_1_2':Parameter(1.7e-15),
              'H_2':Parameter(4.8e-4),
              'Xgs_2': Parameter(7.0e13),
              'Td_2': Parameter(1e-2),
              'w_2':Parameter(3.6e4),
              'T':Parameter(298.0),
              'k_1_2_surf':Parameter(6.0e-12),
             'delta_4':Parameter(1e-7),
             'Db_4':Parameter(1e-10)}


# make the simulate object with the model and parameter dictionary
sim = Simulate(model,param_dict)

# define required parameters
n_layers = 10
rp = 0.2e-4 # radius in cm
time_span = [0,40] # in s (times between which to run the model)
n_time = 999 # number of timepoints to save to output

#spherical V and A
# use simulate.make_layers function
V, A, layer_thick = simulate.make_layers(mod_type,n_layers,rp)

# initial conc. of everything

bulk_conc_dict = {'1':1.21e21,'2':0,'3':0,'4':0} # key=model component number, value=bulk conc
surf_conc_dict = {'1':9.68e13,'2':0,'3':0,'4':0} # key=model component number, value=surf conc

y0 = simulate.initial_concentrations(mod_type,bulk_conc_dict,surf_conc_dict,n_layers) 
    
# now run the model
output = sim.run(n_layers,rp,time_span,n_time,V,A,layer_thick,Y0=y0)

%matplotlib inline
# plot the model
sim.plot()

## Summary

Have a play with the branching ratios in the reaction scheme. You will see different relative amounts of `C` and `D` in the model output. This can be done for many-reaction reaction schemes to increase complexity.

You can also check out the model code outputted as `km-sub_AB_CD.py` and see where the branching ratios are applied in the differential equations defined for `C` and `D`.